In [1]:
#| code-summary: Load Packages
#| code-fold: true

# numerical calculation & data frames
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

# statistics
import statsmodels.api as sm


In [2]:
#| echo: false
from IPython.display import display, HTML
HTML('<style>.output {flex-direction: row;}</style>')
pd.options.display.notebook_repr_html = False

In [3]:
#| code-summary: Options
#| code-fold: true

# pandas options
pd.set_option("mode.copy_on_write", True)
pd.options.display.precision = 2
pd.options.display.float_format = '{:.2f}'.format  # pd.reset_option('display.float_format')
pd.options.display.max_rows = 7

# Numpy options
np.set_printoptions(precision = 2, suppress=True)

## Song rankings for Billboard top 100 in the year 2000
"billboard" in a package "tidyr"  
Source: [The Whitburn Project](https://waxy.org/2008/05/the_whitburn_project/)

다음 링크의 데이터는 빌보드차트에 관한 데이터입니다; [링크](https://www.dropbox.com/s/svkuflje8fesv2a/billboard.csv?dl=1) 


- 각 곡이 차트에 진입한 날짜(`date_entered`)인 첫주(`wk1`)의 순위부터 78주(`wk78`)의 순위까지 기록되어 있습니다. 
- 차트에서 빠진 경우 missing (NA)으로 표시되어 있습니다.  
- 빌보드의 정책과 데이터 추출에 대해서 분명하지 않기 때문에 정확한 분석은 아닐 수 있습니다.
  - 예를 들어, 20주 연속 차트에 있거나, 50위 밖으로 밀려난 경우 차트에서 제거된다고 합니다.

::: {.callout-note}
데이터를 불러오는 여러 방식에 대해서는 교재 참고  
[Chpter 6. Data Loading, Storage, and File Formats](https://wesmckinney.com/book/accessing-data.html) in Python for Data Analysis by Wes McKinney
:::

In [4]:
billboard = pd.read_csv("data/billboard.csv")
billboard.head(5)

         artist                    track date_entered  wk1   wk2   wk3   wk4   
0         2 Pac  Baby Don't Cry (Keep...   2000-02-26   87 82.00 72.00 77.00  \
1       2Ge+her  The Hardest Part Of ...   2000-09-02   91 87.00 92.00   NaN   
2  3 Doors Down               Kryptonite   2000-04-08   81 70.00 68.00 67.00   
3  3 Doors Down                    Loser   2000-10-21   76 76.00 72.00 69.00   
4      504 Boyz            Wobble Wobble   2000-04-15   57 34.00 25.00 17.00   

    wk5   wk6   wk7  ...  wk67  wk68  wk69  wk70  wk71  wk72  wk73  wk74   
0 87.00 94.00 99.00  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  \
1   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2 66.00 57.00 54.00  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3 67.00 65.00 55.00  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4 17.00 31.00 36.00  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   wk75  wk76  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN 

1. 총 몇 명의 가수(`artist`)가 차트에 있으며, 가수별로 몇 곡(`track`)이 차트에 들어있는지 알아보세요. (동명이인은 없다고 가정하고)
2. 곡명은 같지만, 가수가 다른 곡이 있는지 알아보고, 서로 다른 노래가 차트에 몇 개나 있는지 알아보세요.

    살펴보았다면, 이후 grouping시 `artist`와 `track`을 함께 사용해야함을 이해했을 겁니다.

3. 이 데이터를 주(week)에 대해서 아래처럼 long format으로 바꿉니다.
- `wk` column에 `wk1`부터 missing이 없는 `wk*`까지 **숫자**로 표현되고,
- `rank` column에는 해당하는 week의 순위가 나타납니다.
- `melt()`를 이용하고,
- `.str.replace()`와 `.astype("int64")`를 사용해야 할 수 있습니다.
- `.dropna(subset="rank", inplace=True)`를 사용하여 `rank` column의 missing을 처리하세요.
- 결과를 `billboard_long` 변수에 할당하여 이후 문제를 이어가세요.
  - 구하지 못한 경우, 정제된 다음 [파일](https://www.dropbox.com/s/fjepjyl0brr7tlw/billboard_long.csv?dl=1)을 다운받아 사용하세요.

In [5]:
#| echo: false
billboard_long = (
    billboard
    .melt(id_vars=billboard.columns[:3], var_name="wk", value_name="rank")
    .sort_values(["artist", "track"])
)
billboard_long["wk"] = billboard_long["wk"].str.replace("wk", "").astype("int64")
billboard_long.dropna(subset="rank", inplace=True)

In [6]:
billboard_long

                artist                    track date_entered  wk  rank
0                2 Pac  Baby Don't Cry (Keep...   2000-02-26   1 87.00
317              2 Pac  Baby Don't Cry (Keep...   2000-02-26   2 82.00
634              2 Pac  Baby Don't Cry (Keep...   2000-02-26   3 72.00
...                ...                      ...          ...  ..   ...
11728  matchbox twenty                     Bent   2000-04-29  37 38.00
12045  matchbox twenty                     Bent   2000-04-29  38 38.00
12362  matchbox twenty                     Bent   2000-04-29  39 48.00

[5307 rows x 5 columns]

4. 50주 이상(포함) 머무른 곡들을 구해보세요.

- 연속으로 머무를 필요없음; 차트에서 나갔다가 다시 들어오는 곡들이 있음
- 머문 기간을 `.size()`를 이용해 구한 후
- 오래 머무른 순서로 정렬 후
- `query()`를 이용해 50주 이상으로 필터링
- 결과가 `artist`와 `track`순으로 정렬되어 있는지 확인해 주세요!
- 코드는 다음 예처럼 하나로 연결하여 간결하게 구성해보세요.
- 
   ```python
   (
      billboard_long.groupby(...)
      .size()
      ...
      .query(...)
   )   
   ```

   **이후 코드도 최대한 이처럼 간결하게 구성해보도록 하세요.**

5. 4번에서 구한 곡들 각각에 대해서 주에 따라 순위가 어떻게 변화했는지 시각화를 통해 살펴보기 위해서 

- 5.1 먼저, 위 곡들만을 포함하도록 4번의 데이터와 원래 데이터(`billboard_long`)를 `merge()`를 이용해 추린 후, 50주 이상 머문 곡으로 필터링하세요.
  - 4번을  구하지 못한 경우, 다음 [파일](https://www.dropbox.com/s/fo1ous1xmsqiqj9/wks_50.csv?dl=1)을 받아 이용하세요.
- 5.2 `seaborn.objects`를 이용해 **대략** 다음과 같이 주에 따른 순위의 변화를 그려보세요.
   ![](images/plot_ex1.png){width=540}


6. (가수별) 곡별로 차트에 머문 기간(weeks)과 가장 높이 올라간 순위를 구해서, 높은 순위를 달성한 곡일수록 차트에 더 오래 머물었는지 알아봅니다. 

- 6.1 곡별로 차트에 머문 기간을 DataFrame으로 구하고; `.reset_index(name=" ")` 이용
- 6.2 곡별로 최상위 순위를 `min()`을 이용해 DataFrame으로 구한 후
- 6.3 이 두 DataFrame을 `merge()`를 이용해 합친 후
- 6.4 `seaborn.objects`를 이용해 머문 기간에 따른 최상위 순위에 대한 관계를 아래와 같이 scatterplot으로 살펴보세요.
  - 눈에 띄는 점을 간단히 기술해보세요.

   ![](images/plot_ex3.png){width=540}
  
7. 순위 1위를 달성한 곡들에 한해, 차트에 진입시 순위와 1위에 처음 도달한 주(week)의 정보가 다음과 같이 표시되도록 구해보세요.

- 함수를 만들고; `min()`과 `argmin()`이 필요할 수 있음
- `apply()`로 그 함수를 적용하여 구해보세요.

   ```python
   #                 artist                    track  wk  rank
   # 0              Aaliyah                Try Again   1 59.00
   # 1              Aaliyah                Try Again  14  1.00
   # 2  Aguilera, Christina  Come On Over Baby (A...   1 57.00
   # 3  Aguilera, Christina  Come On Over Baby (A...  11  1.00
   # 4  Aguilera, Christina        What A Girl Wants   1 71.00
   # 5  Aguilera, Christina        What A Girl Wants   8  1.00
   ...
   ```

8. 빠르게 1위가 된 곡일 수록 빠르게 차트에서 사라졌을까를 알아보기 위해, 7번의 결과를 이용해 다음과 같이 변형해보세요.

- 7번을 구하지 못한 경우, 다음 [파일](https://www.dropbox.com/s/8lrl7c2u5t34jiv/wk_rank.csv?dl=1)을 받아 이용하세요.

- 즉, 차트 진입시의 순위 정보와, 1위가 된 week의 정보만을 취해, 그 비율(`rate`)를 구하면, 얼마나 빠르게 1위가 되었는지 알 수 있습니다.

   ```python
   #                 artist                    track  wk  rank   rate
   # 0              Aaliyah                Try Again  14 59.00   4.21
   # 1  Aguilera, Christina  Come On Over Baby (A...  11 57.00   5.18
   # 2  Aguilera, Christina        What A Girl Wants   8 71.00   8.88
   # 3        Carey, Mariah  Thank God I Found Yo...  11 82.00   7.45
   # 4                Creed      With Arms Wide Open  27 84.00   3.11
   # 5      Destiny's Child  Independent Women Pa...   9 78.00   8.67
   # ...
   ```
 - 마지막으로, `seaborn.objects`를 이용해 다음과 같이 시각화해보세요.
   ![](images/plot_ex2.png){width=540}
